# Human Bone Fracture Detection

We will detect the type of bone breaks in this project. The plans we will follow is
- ~Make a python virtual environment~
- ~Clean and prepare Data~
- [TODO]Create transfer learning model
- Train and get a decent accuracy using just modifications to head
- Unfreezing and running the model
- Test set accuracy and visualization

In [3]:
!pip install numpy
!pip install pandas
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install matplotlib

Looking in indexes: https://download.pytorch.org/whl/cpu


### Importing dependencies and database

In [1]:
# imorting reqd libs
from pathlib import Path
import pandas as pd
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image


In [23]:
# imoprting dataset


# Creating paths for imports
data_path = Path.cwd()
folder_path = data_path / "bone.v1i.multiclass"

train_path = folder_path / "train"
validation_path = folder_path / "valid"
test_path = folder_path / "test"

train_csv_path = train_path / '_classes.csv'
validation_csv_path = validation_path / '_classes.csv'
test_csv_path = test_path / '_classes.csv'

train_df = pd.read_csv(train_csv_path)
validation_df = pd.read_csv(validation_csv_path)
test_df = pd.read_csv(test_csv_path)

# Removing whiespace before column names
def clean_df(df):
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['fracture', 'Fracture'])
    # df = df.drop(columns=['Fracture'])

    return df

train_df.columns = train_df.columns.str.strip()
validation_df.columns = validation_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Combining columns Fracture and fracture
train_df['Fracture'] = train_df['Fracture'] | train_df['fracture']
train_df = train_df.drop(columns=['fracture'])
train_df = train_df.drop(columns=['Fracture'])
validation_df['Fracture'] = validation_df['Fracture'] | validation_df['fracture']
validation_df = validation_df.drop(columns=['fracture'])
validation_df = validation_df.drop(columns=['Fracture'])
test_df['Fracture'] = test_df['Fracture'] | test_df['fracture']
test_df = test_df.drop(columns=['fracture'])
test_df = test_df.drop(columns=['Fracture'])

# train_df.head()


In [25]:
# creating dataloaders for the images
class BoneDataset(Dataset):
    def __init__(self, folder_path, transform):
        self.folder_path = Path(folder_path)
        self.df = pd.read_csv(self.folder_path / "_classes.csv")
        self.df = clean_df(self.df)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]   # filename
        img_path = self.folder_path / img_name
        image = Image.open(img_path).convert("RGB")

        # label = self.df.iloc[idx, 1]      # class label
        label_row = self.df.iloc[idx, 1:14]
        label = label_row.idxmax()

        if self.transform:
            image = self.transform(image)

        return image, label


# Define transforms
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Create datasets
train_dataset = BoneDataset(folder_path = train_path, transform = transform)
valid_dataset = BoneDataset(folder_path = validation_path, transform = transform)
test_dataset  = BoneDataset(folder_path = test_path, transform = transform)

# Create DataLoaders
batch_size = 64
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
loader_test  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# for images, labels in loader_train:
#     print(labels)
#     break
